# Intro to Random Forest

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline



In [3]:

import fastai

from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [6]:
PATH = "../data/bulldozers/"

In [7]:
!ls {PATH}

Data%20Dictionary.xlsx           TrainAndValid.zip
Machine_Appendix.csv             Valid.7z
Test.csv                         Valid.csv
Train.7z                         Valid.zip
Train.zip                        ValidSolution.csv
TrainAndValid.7z                 median_benchmark.csv
TrainAndValid.csv                random_forest_benchmark_test.csv


In [9]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, 
                     parse_dates=["saledate"])

In [10]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [11]:
display_all(df_raw.tail().T)

401120  \
SalesID                                                             6333336   
SalePrice                                                             10500   
MachineID                                                           1840702   
ModelID                                                               21439   
datasource                                                              149   
auctioneerID                                                              1   
YearMade                                                               2005   
MachineHoursCurrentMeter                                                NaN   
UsageBand                                                               NaN   
saledate                                                2011-11-02 00:00:00   
fiModelDesc                                                           35NX2   
fiBaseModel                                                              35   
fiSecondaryDesc                                                          NX   
fiModelSeries                                                             2   
fiModelDescriptor                                                       NaN   
ProductSize                                                            Mini   
fiProductClassDesc        Hydraulic Excavator, Track - 3.0 to 4.0 Metric...   
state                                                              Maryland   
ProductGroup                                                            TEX   
ProductGroupDesc                                           Track Excavators   
Drive_System                                                            NaN   
Enclosure                                                             EROPS   
Forks                                                                   NaN   
Pad_Type                                                                NaN   
Ride_Control                                                            NaN   
Stick                                                                   NaN   
Transmission                                                            NaN   
Turbocharged                                                            NaN   
Blade_Extension                                                         NaN   
Blade_Width                                                             NaN   
Enclosure_Type                                                          NaN   
Engine_Horsepower                                                       NaN   
Hydraulics                                                        Auxiliary   
Pushblock                                                               NaN   
Ripper                                                                  NaN   
Scarifier                                                               NaN   
Tip_Control                                                             NaN   
Tire_Size                                                               NaN   
Coupler                                                 None or Unspecified   
Coupler_System                                                          NaN   
Grouser_Tracks                                                          NaN   
Hydraulics_Flow                                                         NaN   
Track_Type                                                            Steel   
Undercarriage_Pad_Width                                 None or Unspecified   
Stick_Length                                            None or Unspecified   
Thumb                                                   None or Unspecified   
Pattern_Changer                                         None or Unspecified   
Grouser_Type                                                         Double   
Backhoe_Mounting                                                        NaN   
Blade_Type                                                              NaN   
Travel_Controls                         

In [12]:
display_all(df_raw.describe(include='all').T)    

count unique  \
SalesID                   401125    NaN   
SalePrice                 401125    NaN   
MachineID                 401125    NaN   
ModelID                   401125    NaN   
datasource                401125    NaN   
auctioneerID              380989    NaN   
YearMade                  401125    NaN   
MachineHoursCurrentMeter  142765    NaN   
UsageBand                  69639      3   
saledate                  401125   3919   
fiModelDesc               401125   4999   
fiBaseModel               401125   1950   
fiSecondaryDesc           263934    175   
fiModelSeries              56908    122   
fiModelDescriptor          71919    139   
ProductSize               190350      6   
fiProductClassDesc        401125     74   
state                     401125     53   
ProductGroup              401125      6   
ProductGroupDesc          401125      6   
Drive_System              104361      4   
Enclosure                 400800      6   
Forks                     192077      2   
Pad_Type                   79134      4   
Ride_Control              148606      3   
Stick                      79134      2   
Transmission              183230      8   
Turbocharged               79134      2   
Blade_Extension            25219      2   
Blade_Width                25219      6   
Enclosure_Type             25219      3   
Engine_Horsepower          25219      2   
Hydraulics                320570     12   
Pushblock                  25219      2   
Ripper                    104137      4   
Scarifier                  25230      2   
Tip_Control                25219      3   
Tire_Size                  94718     17   
Coupler                   213952      3   
Coupler_System             43458      2   
Grouser_Tracks             43362      2   
Hydraulics_Flow            43362      3   
Track_Type                 99153      2   
Undercarriage_Pad_Width    99872     19   
Stick_Length               99218     29   
Thumb                      99288      3   
Pattern_Changer            99218      3   
Grouser_Type               99153      3   
Backhoe_Mounting           78672      2   
Blade_Type                 79833     10   
Travel_Controls            79834      7   
Differential_Type          69411      4   
Steering_Controls          69369      5   

                                                                        top  \
SalesID                                                                 NaN   
SalePrice                                                               NaN   
MachineID                                                               NaN   
ModelID                                                                 NaN   
datasource                                                              NaN   
auctioneerID                                                            NaN   
YearMade                                                                NaN   
MachineHoursCurrentMeter                                                NaN   
UsageBand                                                            Medium   
saledate                                                2009-02-16 00:00:00   
fiModelDesc                                                            310G   
fiBaseModel                                                             580   
fiSecondaryDesc                                                           C   
fiModelSeries                                                            II   
fiModelDescriptor                                                         L   
ProductSize                                                          Medium   
fiProductClassDesc        Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...   
state                                                               Florida   
ProductGroup                                                            TEX   
ProductGroupDesc                                           Track Excavators   
Drive_System                                 

In [13]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

In [14]:
m = RandomForestRegressor(n_jobs=-1)
# The following code is supposed to fail due to string values in the input data
m.fit(df_raw.drop('SalePrice', axis=1), df_raw.SalePrice) #Pass in the dependant - to predict variable and 

#df_raw.drop('SalePrice', axis=1) - is all the columns except the SalePrice
#df_raw.SalePrice - this is what we want to predict 

#RandomForestRegressor - this is what I want to use - so I create an object out of it

ValueError: could not convert string to float: 'Conventional'

In [15]:
add_datepart(df_raw, 'saledate')
df_raw.saleYear.head()

0    2006
1    2004
2    2004
3    2011
4    2009
Name: saleYear, dtype: int64

In [ ]:
train_cats(df_raw)

In [ ]:
df_raw.UsageBand.cat.categories
    

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)


In [ ]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes


In [ ]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))


In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

In [ ]:
df_raw = pd.read_feather('tmp/bulldozers-raw')


In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')


In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)


In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000, na_dict=nas)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)


In [ ]:
m = RandomForestRegressor(n_estimators=1, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)


In [ ]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

In [ ]:
preds.shape


In [ ]:
plt.plot([metrics.r2_score(y_valid, np.mean(preds[:i+1], axis=0)) for i in range(10)]);


In [ ]:
m = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=80, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)


In [ ]:
set_rf_samples(20000)


In [ ]:
m = RandomForestRegressor(n_jobs=-1, oob_score=True)
%time m.fit(X_train, y_train)
print_score(m)


In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
reset_rf_samples()


In [ ]:
def dectree_max_depth(tree):
    children_left = tree.children_left
    children_right = tree.children_right

    def walk(node_id):
        if (children_left[node_id] != children_right[node_id]):
            left_max = 1 + walk(children_left[node_id])
            right_max = 1 + walk(children_right[node_id])
            return max(left_max, right_max)
        else: # leaf
            return 1

    root_node_id = 0
    return walk(root_node_id)

In [ ]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
t=m.estimators_[0].tree_


In [ ]:
dectree_max_depth(t)


In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
t=m.estimators_[0].tree_


In [ ]:
dectree_max_depth(t)


In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

In [ ]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)